In [1]:
# This is required to run multiple processes on Unity for some reason.
from multiprocessing import set_start_method
set_start_method('spawn', force=True)


import os
os.environ['XLA_FLAGS'] = '--xla_gpu_enable_command_buffer='
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '0.7'
# os.environ['JAX_PLATFORMS'] = 'cpu'

In [2]:
%matplotlib widget
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import jax
import jax.numpy as jnp
from tqdm.auto import tqdm
from pathlib import Path
from importlib import reload

src = str(Path('../src').resolve())
if src not in sys.path:
    sys.path.append(src)
import config, data, models, train, evaluate

In [4]:
# reload(config)
from config import read_config

cfg, cfg_str = read_config("../runs/Flex_TEALSTM/all_data.yml")
cfg['num_workers'] = 1
cfg['log'] = False
cfg['quiet'] = False
# cfg['batch_size'] = 256


In [5]:
reload(data)
from data import HydroDataset

dataset = HydroDataset(cfg)

Data Hash: 81fdef3f716995354366489caf2cd711a73c299452f099ba372f49ff79067ad9
Using cached basin dataset file.


Updating Indices:   0%|          | 0/504 [00:00<?, ?it/s]

In [6]:
# reload(config)
from config import set_model_data_args
from data import HydroDataLoader

cfg = set_model_data_args(cfg, dataset)

dataloader = HydroDataLoader(cfg, dataset)

Dataloader using 1 parallel CPU worker(s).
Batch sharding set to 1 gpu(s)


In [7]:
reload(train)
from train import Trainer

trainer = Trainer(cfg, dataloader)
# trainer.start_training()

Model contains 376,899 parameters, using 1.44MB memory.


In [38]:
cfg, model, trainer_state, opt_state, _  = train.load_last_state(Path('/work/pi_kandread_umass_edu/tss-ml/runs/Flex_TEALSTM/all_data_20241105_181617'))
cfg['log'] = False
cfg['exclude_target_from_index'] = None
cfg['quiet'] = False
dataset = HydroDataset(cfg)
dataset.update_indices('test')
dataloader = HydroDataLoader(cfg, dataset)

results = evaluate.predict(model, dataloader, return_dt=True, quiet=False, denormalize=True)
len(results)

Loading model state from /work/pi_kandread_umass_edu/tss-ml/runs/Flex_TEALSTM/all_data_20241105_181617/epoch050
Model contains 376,899 parameters, using 1.44MB memory.
Data Hash: 56fb4cb1c9ed67b117f1b80e1685b0410ac31624e2abeca9185b6c5c1f61fb51
Using cached basin dataset file.


Updating Indices:   0%|          | 0/504 [00:00<?, ?it/s]

Updating Indices:   0%|          | 0/126 [00:00<?, ?it/s]

Dataloader using 2 parallel CPU worker(s).
Batch sharding set to 1 gpu(s)


  0%|          | 0/35486 [01:04<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
results

In [35]:
results.index.get_level_values('basin').unique()

Index(['USGS-02342881', 'USGS-09379500', 'USGS-07258000', 'USGS-07344350',
       'USGS-03374100', 'USGS-04192500', 'USGS-07250550', 'USGS-02089500',
       'USGS-05586100', 'USGS-06934500',
       ...
       'USGS-01049265', 'USGS-14321000', 'USGS-08022040', 'USGS-05484500',
       'USGS-06090800', 'USGS-07348500', 'USGS-02375500', 'USGS-12170300',
       'USGS-02225000', 'USGS-08049500'],
      dtype='object', name='basin', length=126)

In [37]:
results

Type                             obs                                   pred  \
Feature                          ssc          flux       usgs_q         ssc   
basin         date                                                            
USGS-02342881 2008-04-07    8.000000           NaN          NaN    6.500932   
USGS-09379500 1990-06-21  461.000092   2030.000366    40.209927  945.776062   
USGS-07258000 1982-09-14   15.999999           NaN          NaN   11.135518   
USGS-07344350 1984-09-11   33.000004           NaN          NaN   27.593639   
USGS-03374100 2011-05-18  132.999985   6999.998535   597.485535   85.348129   
...                              ...           ...          ...         ...   
USGS-09260000 2001-07-09    9.000000           NaN          NaN   48.597797   
USGS-01567000 2016-02-05   85.000015           NaN   611.643982   83.945328   
USGS-05586100 2008-06-01  352.999969           NaN   773.049866  181.221252   
USGS-01198550 1989-04-17  154.000015           NaN          NaN   76.125435   
USGS-04192500 2020-05-20  258.999969  39099.984375  1560.258179  345.981384   

Type                                                  dt          
Feature                           flux       usgs_q era5 landsat  
basin         date                                                
USGS-02342881 2008-04-07    106.486687   170.454849    0       4  
USGS-09379500 1990-06-21   6093.820801    65.676521    0       9  
USGS-07258000 1982-09-14    353.135834   354.905975    0     366  
USGS-07344350 1984-09-11    170.127121    72.581039    0       9  
USGS-03374100 2011-05-18  19989.179688  1967.580444    0       7  
...                                ...          ...  ...     ...  
USGS-09260000 2001-07-09     28.821409     6.182716    0      20  
USGS-01567000 2016-02-05  14058.512695  1385.651978    0      12  
USGS-05586100 2008-06-01  11554.757812   543.344971    0       0  
USGS-01198550 1989-04-17    485.406860   126.301071    0      76  
USGS-04192500 2020-05-20  81495.867188  1891.595459    0       7  

[27008 rows x 8 columns]

In [15]:
key = jax.random.PRNGKey(0)
batch_keys = jax.random.split(key, cfg['batch_size'])

dataloader = HydroDataLoader(cfg, dataset)
for basin, date, batch in tqdm(dataloader):
    batch = trainer.dataloader.shard_batch(batch)
    
    loss, grads, new_model, new_opt_state = train.step.make_step(
        trainer.model, 
        batch,
        batch_keys,
        trainer.opt_state, 
        trainer.optim, 
        trainer.filter_spec, 
        trainer.dataloader.dataset.denormalize_target,
        **trainer.cfg['step_kwargs']
    )
    
    if np.isnan(loss):
        break
    else:
        trainer.model = new_model
        trainer.opt_state = new_opt_state

Dataloader using 1 parallel CPU worker(s).
Batch sharding set to 1 gpu(s)


  0%|          | 0/3735 [00:08<?, ?it/s]

/home/tlanghorst_umass_edu/.conda/envs/tss-ml/lib/python3.12/site-packages/jax/_src/core.py:700: FutureWarning: unhashable type: <class 'jax._src.interpreters.ad.JVPTracer'>. Attempting to hash a tracer will lead to an error in a future JAX release.
  warnings.warn(


In [17]:
np.isnan(loss)

np.False_

In [ ]:
batch = trainer.dataloader.shard_batch(batch)
batch['dynamic_dt']['era5'].device

In [ ]:
cell = trainer.model.encoders['landsat'].cell
cell

In [ ]:
cfg['model_args']['

In [ ]:
x_d = batch['dynamic']['landsat'][0]
x_s = batch['static'][0]
static_bias = trainer.model.static_embedder(x_s, key)

h_0 = jnp.zeros(cfg['model_args']['hidden_size'])
dt = 1

time_weight = cell.time_distance(dt, static_bias)
time_weight.shape

In [ ]:
# lax.scan 
for x in x_d:
    if jnp.all(~jnp.isnan(x)):
        break
x

In [ ]:
gates = jnp.dot(x, cell.weight_ih.T) + jnp.dot(h_0, cell.weight_hh.T) + cell.bias
f, g, o = jnp.split(gates, 3, axis=-1)
i = jax.nn.sigmoid(cell.input_linear(static_bias))

f = jax.nn.sigmoid(f) * time_weight

In [ ]:
import jax.numpy as jnp

params = jnp.split(trainer.model.encoders['landsat'].cell.decay_weights, 2, axis=0)

In [ ]:
params[0,:].shape

In [ ]:
trainer.model.encoders['landsat'].cell

In [ ]:
batch = dataloader.shard_batch(batch)

batch['dynamic_dt']['landsat'].device

In [ ]:
first_values.shape

In [ ]:
last_valid_index.shape

In [ ]:
x_d = batch['dynamic']['landsat']
# x_s = batch['static'][0]



valid_mask = np.all(~np.isnan(x_d),axis=2)

# valid_mask[0] = True

indices = np.arange(valid_mask.shape[1])
                  
valid_indices = np.where(valid_mask, indices, -1)


last_valid_index = np.maximum.accumulate(valid_indices, axis=1) 

first_values = valid_mask[:,0].astype(int)[:, None]
dt = np.concat([first_values, np.diff(last_valid_index, axis=1)],axis=1)


dt[1]


In [ ]:
dt[0]

In [ ]:
plt.close('all')
plt.imshow(dt)#,aspect='auto')

In [ ]:
alt_cfg = cfg.copy()
alt_cfg['batch_size'] = 1
dataloader = TAPDataLoader(alt_cfg, dataset)

for basin, date, batch in dataloader:
    break
    
single_data = {k: v[0,...] for k, v in batch.items()}

In [ ]:
trainer.model(single_data, jax.random.PRNGKey(0))

In [ ]:
batch['y'][0,...].shape

In [ ]:
# Resume training. Either directly from memory or loading a checkpoint.
import optax

# trainer.load_state('epoch100')
# trainer.load_last_state()

more_epochs = 0
new_schedule = optax.exponential_decay(0.01, trainer.epoch+more_epochs, 0.001, transition_begin=trainer.epoch)
trainer.lr_schedule = new_schedule
trainer.num_epochs += more_epochs

#Have to make a new dataloader when the last one is interrupted. 
trainer.dataloader = TAPDataLoader(cfg, dataset) 
trainer.start_training() 

In [ ]:
reload(evaluate)
from evaluate import predict, get_all_metrics

# basin = np.random.choice(dataset.basins).tolist()
basin = 'USGS-09367540'

cfg['data_subset'] = 'predict'
cfg['basin_subset'] =  basin
cfg['num_workers'] = 0 # Faster for small runs
dataloader = TAPDataLoader(cfg, dataset)

results = predict(trainer.model, dataloader, seed=0, denormalize=True)
results['pred'] = results['pred'] * (results['pred']>0) #Clip predictions to 0

results = results.reset_index()
results = results.sort_values(by='date')
results = results.drop(columns=['basin'], axis=1, level=0)
results.set_index('date', inplace=True)


In [ ]:
feature = 'usgs_q'

# Plot the true values and predictions
fig, ax = plt.subplots(figsize=(12, 6))
results['pred'][feature].plot(ax=ax)
results['obs'][feature].plot(ax=ax,linestyle='None',marker='.')

plt.title(f"Basin: {basin}")
plt.legend()
fig.autofmt_xdate()
# plt.ylim([0,20000])
plt.show()

In [ ]:
basin

In [ ]:
"""
'USGS-09367540'
"""

In [ ]:
results.plot.scatter('obs','pred')
plt.gca().axis('square')
# plt.xscale('log')
# plt.yscale('log')
# plt.xlim([0,20])
# plt.ylim([0,20])
plt.show()

In [ ]:
import train
from data import TAPDataset, TAPDataLoader

state_dir = Path("../runs/notebook/20240603_1359/epoch18")
cfg, model, trainer_state, opt_state = train.load_state(state_dir)
dataset = TAPDataset(cfg)

In [ ]:
reload(evaluate)
from evaluate import predict, get_all_metrics

cfg['data_subset'] = 'test'
cfg['num_workers'] = 4
dataloader = TAPDataLoader(cfg, dataset)

results = predict(model, dataloader, seed=0, denormalize=True)
results['pred'] = results['pred']# * (results['pred']>0) #Clip predictions to 0

# results = results.reset_index()
# results = results.sort_values(by='date')

metrics = get_all_metrics(results['obs'],results['pred'])
metrics

In [ ]:
%matplotlib widget
plt.close('all')
plt.scatter(batch['y'][...,-1],pred[...,-1])
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(10,3))
xd = axes[0].imshow(batch['x_dd'][:,:,0],aspect='auto')
fig.colorbar(xd, ax=axes[0])
xs = axes[1].imshow(batch['x_s'],aspect='auto')
fig.colorbar(xs, ax=axes[1]) 

In [ ]:
batch['x_dd'][:,:,0].shape

In [ ]:
basins[idx_max_err]

In [ ]:
positional_encoding = trainer.model.d_encoder.embedder.positional_encoding

plt.figure(figsize=(10, 8))
plt.imshow(positional_encoding, cmap='viridis')
plt.xlabel('Embedding Dimension')
plt.ylabel('Position')
plt.title('Positional Encodings')
plt.show()